In [4]:
class Yeet(type):
    def __init__(self, *args, **kwargs):
        print("meta init")
        super().__init__(*args, **kwargs)

In [6]:
class Yote(metaclass=Yeet):
    def __init__(self):
        print("class init")
        
Yote()

meta init
class init
